# OWL Win Predictor Data Processing
## Let's dive deeper.

---

This file was created after I turned in the original data processing file.
 
In this file, I would like to explore the data more deeply without accidentally altering the data processing that has already been done.

I am also going to be experimenting with altair graphing.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import datetime
import pickle
from joblib import dump, load 
import altair as alt

In [12]:
player_stats = pd.concat(map(pd.read_csv, glob.glob('datasets/phs*/*.csv')))
player_stats.head()

player_stats.drop(["pelstart_time"], axis = 1, inplace = True)

# Resets index and drops the new row it makes
player_stats.reset_index(drop = True, inplace = True)

player_stats['year'] = pd.to_datetime(player_stats['start_time'])
player_stats['year'] = pd.DatetimeIndex(player_stats["year"]).year

# sorts by match_id in ascending order and resets the index
player_stats = player_stats.sort_values(by = ["match_id", "map_name", "team" ], ascending = True)

In [13]:
player_stats
# Can I pivot the values so each hero has a column for the heroes with the stat name?

,start_time,match_id,stage,map_type,map_name,player,team,stat_name,hero,stat_amount,year
55256,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,All Damage Done,All Heroes,18079.206920,2018.0
55257,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Assists,All Heroes,17.000000,2018.0
55258,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Barrier Damage Done,All Heroes,2893.659185,2018.0
55259,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Damage - Quick Melee,All Heroes,443.204010,2018.0
55260,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Deaths,All Heroes,11.000000,2018.0
...,...,...,...,...,...,...,...,...,...,...,...
3921522,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Time Holding Ultimate,D.Va,101.423003,2021.0
3921523,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Time Played,D.Va,424.380640,2021.0
3921524,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Ultimates Earned - Fractional,D.Va,3.763763,2021.0
3921525,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Ultimates Used,D.Va,2.000000,2021.0


In [14]:
map_stats = pd.read_csv("datasets\match_map_stats.csv")

# Add the year column
map_stats['year'] = pd.to_datetime(map_stats['round_start_time'])
map_stats['year'] = pd.DatetimeIndex(map_stats["year"]).year
map_stats

,round_start_time,round_end_time,stage,match_id,game_number,match_winner,map_winner,map_loser,map_name,map_round,...,team_two_name,attacker_payload_distance,defender_payload_distance,attacker_time_banked,defender_time_banked,attacker_control_perecent,defender_control_perecent,attacker_round_end_score,defender_round_end_score,year
0,2018-01-11 00:12:07,2018-01-11 00:20:07,Overwatch League - Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,1,...,San Francisco Shock,75.615051,0.000000,0.000000,240.000000,NaN,NaN,2,0,2018.0
1,2018-01-11 00:22:05,2018-01-11 00:27:59,Overwatch League - Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,2,...,San Francisco Shock,75.649597,75.615051,125.750572,0.000000,NaN,NaN,3,2,2018.0
2,2018-01-11 00:34:39,2018-01-11 00:38:29,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,1,...,San Francisco Shock,0.000000,0.000000,250.492004,240.000000,NaN,NaN,2,0,2018.0
3,2018-01-11 00:40:27,2018-01-11 00:44:41,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,2,...,San Francisco Shock,0.000000,0.000000,225.789032,250.492004,NaN,NaN,2,2,2018.0
4,2018-01-11 00:46:09,2018-01-11 00:49:48,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,3,...,San Francisco Shock,0.000000,0.000000,36.396057,250.492004,NaN,NaN,4,2,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,2021-04-25 22:18:17,2021-04-25 22:21:47,OWL 2021,37219,3,Washington Justice,Vancouver Titans,Washington Justice,Temple of Anubis,2,...,Washington Justice,0.000000,0.000000,260.375977,60.000000,NaN,NaN,2,2,2021.0
9323,2021-04-25 22:23:13,2021-04-25 22:24:29,OWL 2021,37219,3,Washington Justice,Vancouver Titans,Washington Justice,Temple of Anubis,3,...,Washington Justice,0.000000,0.000000,0.000000,260.375977,NaN,NaN,2,2,2021.0
9324,2021-04-25 22:25:55,2021-04-25 22:28:11,OWL 2021,37219,3,Washington Justice,Vancouver Titans,Washington Justice,Temple of Anubis,4,...,Washington Justice,0.000000,0.000000,124.516968,0.000000,NaN,NaN,3,2,2021.0
9325,2021-04-25 22:40:43,2021-04-25 22:47:29,OWL 2021,37219,4,Washington Justice,Washington Justice,Vancouver Titans,Blizzard World,1,...,Washington Justice,111.936325,0.000000,0.000000,0.000000,NaN,NaN,1,0,2021.0


In [30]:
# Merges map stats and player stats with an outer join on the keys match_id, map_name, and year.
merge_all = pd.merge(player_stats, map_stats, how = "left", on = ["match_id", "map_name", "year"])

# Sorts the values by match_id, map_name, and team
merge_all = merge_all.sort_values(by = ["match_id", "map_name", "team" ], ascending = True)

merge_all.reset_index(drop = True, inplace = True)

In [31]:
merge_all.loc[merge_all["match_winner"] == merge_all["team"], 'match_win'] = "win"
merge_all.loc[merge_all["match_winner"] != merge_all["team"], 'match_win'] = "lose"

merge_all.loc[merge_all["map_winner"] == merge_all["team"], 'map_win'] = "win"
merge_all.loc[merge_all["map_winner"] != merge_all["team"], 'map_win'] = "lose"

In [ ]:
merge_all.loc[merge_all["match_win"] == "win", 'match_win_bin'] = 1
merge_all.loc[merge_all["match_win"] == "lose", 'match_win_bin'] = 0

merge_all.loc[merge_all["map_winner"] == merge_all["team"], 'map_win'] = "win"
merge_all.loc[merge_all["map_winner"] != merge_all["team"], 'map_win'] = "lose"

In [32]:
merge_all

,start_time,match_id,stage_x,map_type,map_name,player,team,stat_name,hero,stat_amount,...,attacker_payload_distance,defender_payload_distance,attacker_time_banked,defender_time_banked,attacker_control_perecent,defender_control_perecent,attacker_round_end_score,defender_round_end_score,match_win,map_win
0,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,All Damage Done,All Heroes,18079.206920,...,75.615051,0.000000,0.000000,240.00000,NaN,NaN,2.0,0.0,win,win
1,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,All Damage Done,All Heroes,18079.206920,...,75.649597,75.615051,125.750572,0.00000,NaN,NaN,3.0,2.0,win,win
2,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Assists,All Heroes,17.000000,...,75.615051,0.000000,0.000000,240.00000,NaN,NaN,2.0,0.0,win,win
3,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Assists,All Heroes,17.000000,...,75.649597,75.615051,125.750572,0.00000,NaN,NaN,3.0,2.0,win,win
4,1/11/2018 0:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Barrier Damage Done,All Heroes,2893.659185,...,75.615051,0.000000,0.000000,240.00000,NaN,NaN,2.0,0.0,win,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9694320,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Ultimates Earned - Fractional,D.Va,3.763763,...,0.000000,0.000000,0.000000,268.27002,NaN,NaN,0.0,2.0,win,win
9694321,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Ultimates Used,D.Va,2.000000,...,0.000000,0.000000,268.270020,0.00000,NaN,NaN,2.0,0.0,win,win
9694322,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Ultimates Used,D.Va,2.000000,...,0.000000,0.000000,0.000000,268.27002,NaN,NaN,0.0,2.0,win,win
9694323,2021-04-17 10:23:55,37256,OWL 2021,ASSAULT,Temple of Anubis,Void,Shanghai Dragons,Weapon Accuracy,D.Va,0.310919,...,0.000000,0.000000,268.270020,0.00000,NaN,NaN,2.0,0.0,win,win
